In [8]:
import datetime as dt
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from scipy.special import rel_entr
import random #GGC added library

In [2]:
for k in range(8):
    print(k)

0
1
2
3
4
5
6
7


# Fixed parameters

In [5]:
#number of parameter vectors
samplesize = 5000

#number of bins applied to build the histogram in the calculation of expressibility
nbins = 75

#width of the bins, considering that the fidelity values goes from 0 to 1
width = 1.0/nbins

# Variable parameters (qubits and layers)

In [6]:
#List of number of qubits
qubits = [4]
#qubits = [8]

#List of number of layers
#layers = [50, 150, 300, 450, 500]
layers = [int(20/8), int(30/8), int(40/8), int(60/8), int(70/8), int(80/8), int(90/8), int(100/8), int(110/8)] #/x porque agora temos x gates por layer no no-connection
#layers = [int(30/16), int(60/16), int(90/16), int(120/16), int(180/16), int(210/16), int(240/16),int(270/16), int(300/16), int(330/16)]

# Fidelity histogram and Lorenz curves

In [5]:
def fidel_histogram(binsnumb, samples, circuit, nlayers):
    
    fidel_vector = []
    CumAvg = []
    DCum = []
    
    CumAvg=0
    DCum=0
    
    for i in range(samples):
        
        #Here the parameter vectors are applied to generate two states considering a particular circuit
        #structure that is given as an input for the function.
            
        state1 = circuit()
        state2 = circuit()
            
        #Fidelity computation
            
        F = np.abs( np.dot(state1, state2) )**2
        
        fidel_vector.append(F)

        #1st Section added by GGC, SD of Lorenz curves, in samples loop, using final vector/each circuit set

        P = np.abs( state1 )**2

        #Decreasing ordering of Pcomp
        P[::-1].sort()
	
        #Cummulants calculation
        Cum=np.cumsum(P)

        #Cummulants average and squared average calculation
        CumAvg=CumAvg+Cum
        DCum=DCum+Cum**2

        #End of 1st Section added by GGC, SD of Lorenz curves

        
        
    fidel_vector = np.asarray(fidel_vector)
    # is this ok?
    # CumAvg = np.asarray(CumAvg)
    # DCum = np.asarray(DCum)

    binsize = 1.0/binsnumb
    bins = np.arange(0, 1. + binsize, binsize)
        
    #Generation of the histograms to compute D_{KL}
        
    histogram, bins = np.histogram(fidel_vector, bins=bins)[0]/samples, np.histogram(fidel_vector, bins=bins)[1]


        
    #2nd Section added by GGC, SD of Lorenz curves final computation at the end of samples loop

    SDL=np.sqrt(np.abs(DCum/samples-(CumAvg/samples)**2))

    #2nd Section added by GGC, SD of Lorenz curves final computation at the end of samples loop
        
    return histogram, bins, SDL

# Code that executes the different conditions

In [6]:
i1 = [] #GGC not sure about the need of this defintion


now0 = dt.datetime.now() #monitorar tempo de início
now0 = now0.strftime("%Y-%m-%d %H:%M:%S")
print("Tempo inicial: ")
print(now0)
print()
    
for wires in qubits:
    #This command initializes the quantum circuit of qubits in pennylane, with number of qubits given by wires
    #and considering analytical computations (shots=None), if we want to compute mean values of observables.
    #shots different from None would lead to a stochastic calculation of mean values.
        
    dev = qml.device("default.qubit", wires=wires, shots=None)
        
#ATUALIZACAO: 04/07/2025====================================================================================================================        
    for nlayers in layers:

        #GGC addition of random G3/G2 circuit (I have left the input parameter (though useless) in order to keep code shape

        def layer_random():
            for index in range(8):
                i1 = random.sample(range(0,wires), 2)
    
                i2 = random.randint(1,2)
                if   i2 == 1 :
                        qml.Hadamard(wires=[i1[0]])
                elif i2 == 2 :
                        qml.S(wires=[i1[0]]) #Other option: qml.T(wires=[i1[0]])
                    
            #CNOTS(Diferentes topologias)

            #qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain") #linear

            #qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain") #ring
            #qml.CNOT(wires=[wires-1,0])

            #for i in range(wires-1): #star
            #    qml.CNOT(wires = [0, i+1])
        
#============================================================================================================================================        
        def randomC():
            qml.layer(layer_random, nlayers)
            return qml.state()

        random_circuit = qml.QNode(randomC, dev)

        #End of GGC addition of random G3 circuit
        
        random_hist, bins, SDL_random = fidel_histogram(nbins, samplesize, random_circuit, nlayers)
        
        #Haar histogram for 'wires' qubits
        N = 2**wires #dimension of the Hilbert space
        histogram_Haar = []
        for i in range(nbins):
            histogram_Haar.append(  (1-bins[i])**(N-1) - (1-bins[i+1])**(N-1)  )

        #GGC Random G3 circuit addition
            
        D_KL_random = sum( rel_entr(random_hist, histogram_Haar) )
        
        f = open("expressibility_LorenzCurves_G2noConnections_qubits{0}_gates{1}_samples{2}.txt"
                     .format(wires,nlayers,samplesize), "w")
        
        print('Circuit,Expressibility,StDevLorenz', file = f)

        #GGC Random G3 circuit addition
            
        print('Random Circuit G3,', D_KL_random,',',SDL_random, file = f)
        
        print('Execution: qubits=',wires,'; layers=', nlayers)
        f.close()
        now1 = dt.datetime.now() #monitorar tempo de início
        now1 = now1.strftime("%Y-%m-%d %H:%M:%S")
        print("Tempo: ",now1)
        print()
        
now1 = dt.datetime.now() #monitorar tempo de início
now1 = now1.strftime("%Y-%m-%d %H:%M:%S")

f.close()

print()
print("Tempo inicial: ")
print(now0)
print("Tempo final:")
print(now1)

Tempo inicial: 
2025-07-10 13:13:27

Execution: qubits= 4 ; layers= 2
Tempo:  2025-07-10 13:13:38

Execution: qubits= 4 ; layers= 3
Tempo:  2025-07-10 13:13:52

Execution: qubits= 4 ; layers= 5
Tempo:  2025-07-10 13:14:11

Execution: qubits= 4 ; layers= 7
Tempo:  2025-07-10 13:14:40

Execution: qubits= 4 ; layers= 8
Tempo:  2025-07-10 13:15:13

Execution: qubits= 4 ; layers= 10
Tempo:  2025-07-10 13:15:53

Execution: qubits= 4 ; layers= 11
Tempo:  2025-07-10 13:16:35

Execution: qubits= 4 ; layers= 12
Tempo:  2025-07-10 13:17:21

Execution: qubits= 4 ; layers= 13
Tempo:  2025-07-10 13:18:10


Tempo inicial: 
2025-07-10 13:13:27
Tempo final:
2025-07-10 13:18:10
